In [2]:
import mne
import numpy as np
import os
from pathlib import Path
from EEG_quality_assessment.signal_metrics import SignalMetrics
import asrpy
import pyprep
import shutil

from eeg_fmri_cleaning.utils import read_raw_eeg
from eeg_fmri_cleaning.main import (
    clean_gradient,
    clean_bcg
)
import bids
from bids.layout import BIDSLayout, BIDSFile

In [3]:
f = BIDSFile('projects/EEG_FMRI/bids_eeg/BIDS/NEW/RAW/sub-01/ses-01/eeg/sub-01_ses-01_task-rest_eeg.fif')

In [8]:
import os
import tempfile
import json



Temporary BIDS dataset created at /var/folders/l3/myr9vj8933q4vwhkbvxcx6gc0000gn/T/tmpem394tqi


In [12]:
from bids import BIDSLayout

# Initialize a BIDS Layout with the path of the temp BIDS directory
layout = BIDSLayout(temp_bids_path, validate=True)

# Example operation: get BIDS entities
e = layout.get_entities()

In [16]:
def create_temp_bids_eeg(n_subjects: int = 1, n_sessions: int = 1, n_runs: int = 1):
    """
    Create a dummy BIDS dataset for EEG data with multiple subjects, sessions, and runs.

    Args:
        n_subjects (int, optional): Number of subjects to create. Defaults to 1.
        n_sessions (int, optional): Number of sessions per subject. Defaults to 1.
        n_runs (int, optional): Number of runs per session. Defaults to 1.

    Returns:
        str: The path of the temporary BIDS dataset.
    """
    # Create a temporary directory for the BIDS dataset
    temp_dir = tempfile.mkdtemp()

    # Define the necessary BIDS files for dataset description
    dataset_description = {
        "Name": "THIS IS A DUMMY DATASET",
        "BIDSVersion": "1.4.0",
        "License": "CC0",
        "Authors": ["Jane Doe", "John Doe"]
    }

    # Write dataset_description.json
    with open(os.path.join(temp_dir, "dataset_description.json"), 'w') as desc_file:
        json.dump(dataset_description, desc_file, indent=4)

    # Initialize participants data
    participants_data = "participant_id\tage\n"

    # Generate subjects, sessions, and runs
    for sub_id in range(1, n_subjects + 1):
        subject_label = f"sub-{sub_id:02d}"
        participants_data += f"{subject_label}\t{20 + sub_id}\n"  # Just example ages

        for ses_id in range(1, n_sessions + 1):
            session_label = f"ses-{ses_id:02d}"
            
            for run_id in range(1, n_runs + 1):
                run_label = f"run-{run_id:02d}"
                eeg_dir = os.path.join(temp_dir, subject_label, session_label, "eeg")
                os.makedirs(eeg_dir, exist_ok=True)  # Create necessary directories

                # Define file names for EEG data files
                eeg_filenames = [
                    f"{subject_label}_{session_label}_task-test_{run_label}_eeg.vhdr",
                    f"{subject_label}_{session_label}_task-test_{run_label}_eeg.eeg",
                    f"{subject_label}_{session_label}_task-test_{run_label}_eeg.vmrk"
                ]

                # Create dummy EEG files
                for eeg_file in eeg_filenames:
                    with open(os.path.join(eeg_dir, eeg_file), 'w') as file:
                        file.write("Dummy EEG file content\n")

    # Write participants.tsv
    with open(os.path.join(temp_dir, "participants.tsv"), 'w') as part_file:
        part_file.write(participants_data)

    print(f"Temporary BIDS dataset created at {temp_dir}")
    return temp_dir
temp_dir = create_temp_bids_eeg(n_subjects=2, n_sessions=2, n_runs=2)

Temporary BIDS dataset created at /var/folders/l3/myr9vj8933q4vwhkbvxcx6gc0000gn/T/tmpw7753x24


In [20]:
temp_dir

'/var/folders/l3/myr9vj8933q4vwhkbvxcx6gc0000gn/T/tmpw7753x24'

In [17]:

layout = BIDSLayout(temp_dir)

In [19]:
layout.get_files()

{'/var/folders/l3/myr9vj8933q4vwhkbvxcx6gc0000gn/T/tmpw7753x24/dataset_description.json': <BIDSJSONFile filename='/var/folders/l3/myr9vj8933q4vwhkbvxcx6gc0000gn/T/tmpw7753x24/dataset_description.json'>,
 '/var/folders/l3/myr9vj8933q4vwhkbvxcx6gc0000gn/T/tmpw7753x24/participants.tsv': <BIDSDataFile filename='/var/folders/l3/myr9vj8933q4vwhkbvxcx6gc0000gn/T/tmpw7753x24/participants.tsv'>,
 '/var/folders/l3/myr9vj8933q4vwhkbvxcx6gc0000gn/T/tmpw7753x24/sub-01/ses-02/eeg/sub-01_ses-02_task-test_run-01_eeg.eeg': <BIDSFile filename='/var/folders/l3/myr9vj8933q4vwhkbvxcx6gc0000gn/T/tmpw7753x24/sub-01/ses-02/eeg/sub-01_ses-02_task-test_run-01_eeg.eeg'>,
 '/var/folders/l3/myr9vj8933q4vwhkbvxcx6gc0000gn/T/tmpw7753x24/sub-01/ses-02/eeg/sub-01_ses-02_task-test_run-02_eeg.eeg': <BIDSFile filename='/var/folders/l3/myr9vj8933q4vwhkbvxcx6gc0000gn/T/tmpw7753x24/sub-01/ses-02/eeg/sub-01_ses-02_task-test_run-02_eeg.eeg'>,
 '/var/folders/l3/myr9vj8933q4vwhkbvxcx6gc0000gn/T/tmpw7753x24/sub-01/ses-02/eeg/sub

In [22]:
a_list = list()
a_list.append("GRAD")
print(a_list)
len(a_list)

['GRAD']


1

In [8]:
from utils import dummy_bids
from bids.layout import BIDSLayout
dummy = dummy_bids.DummyDataset(n_runs = 2)
dummy.create_eeg_dataset()
layout = BIDSLayout(dummy.bids_path)
files = layout.get(extension='.vhdr')
#dummy.flush(check=False)

Temporary BIDS dataset created at /var/folders/l3/myr9vj8933q4vwhkbvxcx6gc0000gn/T/tmpuq4l5t08/RAW


In [13]:
import mne
info = mne.create_info(32, 1000)
raw = mne.simulation.simulate_raw(info)
raw.plot()

ValueError: src must be a string or instance of SourceSpaces, got None (type <class 'NoneType'>)

In [7]:
dummy.flush(check=False)

Removing the temporary directory...
Content being removed:
/var/folders/l3/myr9vj8933q4vwhkbvxcx6gc0000gn/T/tmp8tvysj1_/
└── RAW/
    ├── sub-01/
    │   └── ses-01/
    │       └── eeg/
    │           ├── sub-01_ses-01_task-test_run-01_eeg.eeg
    │           ├── sub-01_ses-01_task-test_run-01_eeg.json
    │           ├── sub-01_ses-01_task-test_run-01_eeg.vhdr
    │           ├── sub-01_ses-01_task-test_run-01_eeg.vmrk
    │           ├── sub-01_ses-01_task-test_run-02_eeg.eeg
    │           ├── sub-01_ses-01_task-test_run-02_eeg.json
    │           ├── sub-01_ses-01_task-test_run-02_eeg.vhdr
    │           └── sub-01_ses-01_task-test_run-02_eeg.vmrk
    ├── dataset_description.json
    └── participants.tsv
The tree was successfully removed.


In [4]:
import utils.simulated_data as sim

In [6]:
sim.DirectoryTree()

TypeError: DirectoryTree.__init__() missing 1 required positional argument: 'root_dir'

In [38]:
l = list(layout.get_files())

In [1]:
a = {'a':1, 'b':2}
for i in a:
    print(i)

a
b


In [30]:
from pathlib import Path
import os
filename = Path('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/RAW/sub-01/ses-01/eeg/sub-01_ses-01_task-rest_eeg.fif')
os.path.splitext(filename)

('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/RAW/sub-01/ses-01/eeg/sub-01_ses-01_task-rest_eeg',
 '.fif')

In [19]:
import bids
filename = 'sub-01_ses-01_task-rest_eeg.fif'
file = bids.utils.make_bidsfile('sub-01_ses-01_task-rest_eeg.fif')

In [15]:
bids.layout.parse_file_entities(file)

{'session': '01', 'task': 'rest', 'suffix': 'eeg', 'extension': ".fif'>"}

In [21]:
test = bids.layout.BIDSFile(filename)

In [10]:
import mne
import numpy as np
base_filename = 'sub-01_ses-01_task-rest_eeg.set'
random_data = np.random.rand(32, 1000)
ch_names = [f'EEG {i}' for i in range(32)]
info = mne.create_info(ch_names, 1000)
raw = mne.io.RawArray(random_data, info)
mne.export.export_raw(base_filename, raw, fmt='eeglab', overwrite=True)

Creating RawArray with float64 data, n_channels=32, n_times=1000
    Range : 0 ... 999 =      0.000 ...     0.999 secs
Ready.


In [4]:
import pybv
data = np.random.random((3, 5))
# write data with varying units
# Note channels A1 and A2 are expected to be in volt and will get
# rescaled to µV and mV respectively.
# TEMP is expected to be in some other unit (i.e., NOT volt), and
# will not get scaled (it is written "as is")
pybv.write_brainvision(data=data, sfreq=1, ch_names=["A1", "A2", "TEMP"],
                       folder_out="./",
                  fname_base="pybv_test_file",
                  unit=["µV", "mV", "°C"])

/opt/anaconda3/envs/mne/lib/python3.12/site-packages/pybv/io.py:681: UserWarning: Encountered unsupported voltage units: mV
We will scale the data appropriately, but for maximum compatibility you should use µV for all channels.
  warn(msg)
/opt/anaconda3/envs/mne/lib/python3.12/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: °C
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [11]:
a = ['a','b','c']
b = [False, True, False]
print(a[b])

TypeError: list indices must be integers or slices, not list

In [25]:
os.path.isfile(filename_base + '.json')

True

In [5]:
mne.channels.get_builtin_montages()

['standard_1005',
 'standard_1020',
 'standard_alphabetic',
 'standard_postfixed',
 'standard_prefixed',
 'standard_primed',
 'biosemi16',
 'biosemi32',
 'biosemi64',
 'biosemi128',
 'biosemi160',
 'biosemi256',
 'easycap-M1',
 'easycap-M10',
 'easycap-M43',
 'EGI_256',
 'GSN-HydroCel-32',
 'GSN-HydroCel-64_1.0',
 'GSN-HydroCel-65_1.0',
 'GSN-HydroCel-128',
 'GSN-HydroCel-129',
 'GSN-HydroCel-256',
 'GSN-HydroCel-257',
 'mgh60',
 'mgh70',
 'artinis-octamon',
 'artinis-brite23',
 'brainproducts-RNP-BA-128']

In [ ]:
file_list = layout.get(subject='01', 
           extension='set')
derivative_path = Path('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/DERIVATIVES')

def is_task_checker(BIDSFile):
    return 'checker' in BIDSFile.get_entities()['task']

def report(message, filename):
    with open(filename, 'a') as f:
        f.write(message)
        f.write('\n')

def cbin_cleaner(file):
    raw = read_raw_eeg(file.path)
    if file.get_entities()['task'] == 'checker':
        raw = clean_gradient(raw)
    raw = clean_bcg(raw)
    return raw

def asr_cleaner(raw):
    asr = asrpy.ASR()
    asr.fit(raw)
    raw = asr.transform(raw)
    return raw

def copy_sidecar(file, where_to_copy):
    filename_base, filename_extension = os.path.splitext(file.path)
    sidecar_filename = filename_base + '.json'
    if os.path.isfile(sidecar_filename):
        shutil.copyfile(sidecar_filename, where_to_copy)
     
def make_directory(file, derivative_path):
    entities = bids.layout.parse_file_entities(file)
    file_path = Path(derivative_path)
    file_path.joinpath(entities['subject'],
                       entities['session'],
                       entities['datatype'])
    file_path.mkdir(parents=True, exist_ok=True)
    
def pyprep_cleaner(raw):
    montage = mne.channels.make_standard_montage('easycap-M10')
    raw.apply_montage(montage)
    prep_params = {
        "ref_chs": "eeg",
        "reref_chs": "eeg",
        "line_freqs": np.arange(60, raw.info["sfreq"]/ 2, 60),
    }
    prep = pyprep.PrepPipeline(raw,
                               prep_params, 
                               channel_wise=True)
    prep.fit()
    raw = prep.transform(raw)
    return raw
        

In [ ]:
for file in file_list:
    if is_task_checker(file):
        try:
            raw = cbin_cleaner(file)
            
            
            pyprep_cleaner(raw)
            asr_cleaner(raw)
            report(file.path,
                f"{derivative_path}/processed.txt")
        except Exception as e:
            report(f"{file.path}: {str(e)}", 
                f"{derivative_path}/not_processed.txt")
            continue

In [3]:
a = (1,2)
try:
    a[0] = 3
except Exception as e:
    print(e)

'tuple' object does not support item assignment
